In [3]:
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
import re
import json

In [13]:
file_path_to_json = ''
files = json.load(open(file_path_to_json,'r'))
pd.set_option('display.max_columns',150)
pd.set_option('display.max_rows',150)

In [29]:

X_ds = pd.read_parquet(files['Prepro']['Train_All'])


X = pd.read_parquet(files['Prepro']['Train_All'])


X_test = pd.read_parquet(files['Prepro']['Test_All'])


test = pd.read_parquet(files['Prepro']['Test_All'])

In [31]:

test.shape, X.shape

((56924, 61), (293218, 61))

In [ ]:
X_what.shape, test.shape, X.shape, X_ds.shape

((293218, 119), (56924, 61), (293218, 119), (293218, 61))

In [8]:
list(X.columns)

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'DOT_ID_Reporting_Airline',
 'IATA_CODE_Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityD

In [32]:
## Script to transform dataset

# List of flight-related columns.
flight_cols = ['Year', 'Quarter', 'Month',
                  'DayofMonth', 'DayOfWeek','OriginAirportSeqID',
                  'DestAirportSeqID', 'Dest','DestStateFips',
                  'CRSElapsedTime', 'Distance',
                  'DistanceGroup']


def combine_mfr(df):
  X = df.copy()

  # This function maps to a consolidated list of manufacturers return a new DF
  X['mfr'] = df['mfr'].map({'MCDONNELL DOUGLAS':'MCDONNELL',
                            'BOEING': 'BOEING',
                            'MCDONNELL DOUGLAS AIRCRAFT CO': 'MCDONNELL',
                            'AIRBUS': 'AIRBUS','AIRBUS INDUSTRIE': 'AIRBUS',
                            'MCDONNELL DOUGLAS CORPORATION': 'MCDONNELL'
                            })

  return X


# def combine_date_time(df):
#   X = df.copy()
#   X.rename(columns={'DayofMonth':'Day'},inplace=True)
#   X['Date']=pd.to_datetime(X[['Year', 'Month', 'Day']])
#   X.drop(['Month', 'Day','Quarter','DayOfWeek','Year'], axis=1,inplace=True)
#   return X





def clean_column_names(df):
  # This cleans-up and renames the feature union's prefixes for each column
  # it first drops the categorical Dest column (which was preserved from the cols_tfr)
  X = df.copy()
  X.drop('feat__Dest',axis=1,inplace=True)
  ins_inc = 0
  new_cols = []
  cols_to_remove = []
  for col in X.columns:
      if '_99' in col:
        cols_to_remove.append(col)
      else:
        new_col = re.sub('Ordinal__','',col)
        new_col = re.sub('feat__','',new_col)
        X.rename({col: new_col},inplace=True,axis=1)
  X.drop(cols_to_remove,axis=1,inplace=True)

  # this next line basically reorders the columns so that the df starts with flight cols
  order_cols = flight_cols + [col for col in X.columns if col not in flight_cols]
  return X[order_cols]


# In order to leverage this for fit/transform we need to wrap into a Function Transformer
mfr_consolidate = (FunctionTransformer(combine_mfr)
                  .set_output(transform='pandas'))

column_cleaner = (FunctionTransformer(clean_column_names)
                  .set_output(transform='pandas'))


def replace_9s(df):
    X = df.copy()
    dub_cols = ['liq_precip_qty_d']
    trip_cols = ['w_dir_angle', 'w_dir_angle_d']
    quad_cols = ['w_speed_rate','liq_precip_dim','w_speed_rate_d','liq_precip_dim_d']
    quin_cols = ['sky_c_hgt','sea_lvl_p','sky_cov_base_hgt','sky_sum_hgt','sky_low_cld_base_hgt',
                     'at_pres_altimeter_rate','at_pres_stn_rate','sky_c_hgt_d', 'sea_lvl_p_d','sky_cov_base_hgt_d',
                     'sky_sum_hgt_d','sky_low_cld_base_hgt_d','at_pres_altimeter_rate_d','at_pres_stn_rate_d']
    sext_cols = ['vis_dist','vis_dist_d']


    for col in dub_cols:
        X.replace({col:{99:-1}},inplace=True)
    for col in trip_cols:
        X.replace({col:{999:-1}},inplace=True)
    for col in quad_cols:
        X.replace({col:{9999:-1}},inplace=True)
    for col in quin_cols:
        X.replace({col:{99999:-1}},inplace=True)
    for col in sext_cols:
        X.replace({col:{999999:-1}},inplace=True)
    print('Shape of df:', df.shape)
    for col in ['tmp_air', 'tmp_air_d', 'tmp_dew', 'tmp_dew_d']:
        col_df = df[df[col] != 9999][col]
        print('Shape of ' + col + ':', col_df.shape)
        col_mean = col_df.mean()
        print('Average of ' + col + ':', col_df.mean())
        X.replace({col:{9999:col_mean}},inplace=True)
    return X


mis_replace = FunctionTransformer(replace_9s).set_output(transform='pandas')


dest_tfr = ColumnTransformer(transformers=[('dests',
                                            OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=999),
                                             ['Dest']
                                            )],
                             remainder='drop',
                             verbose_feature_names_out=False)

#
mfr_trf = ColumnTransformer(transformers=[('mfr',
                                           Pipeline([('mfr_consolidate', mfr_consolidate),
                                                     ('mfr_ohe',OneHotEncoder(sparse_output=False)
                                                     )]),
                                                    ['mfr']
                                                    )],
                             remainder='drop',
                             verbose_feature_names_out=False).set_output(transform='pandas')


# each column listed is passed to OneHotEncoding to be split into several cols
cols_tfr = ColumnTransformer([('columns_encoded',
                              OneHotEncoder(sparse_output=False),
                              ['Reporting_Airline',
                                'mfr',
                              'engine_type',
                                'w_type',
                                'sky_c_det',
                                'sky_cov',
                                'sky_obs_tot_cov',
                                'w_type_d',
                                'sky_c_det_d',
                                'sky_cov_d',
                                'sky_obs_tot_cov_d']
                                )],
                remainder='passthrough',
                verbose_feature_names_out=False).set_output(transform='pandas')


# The column transformers need to be joined together to recreate the dataframe
feat_union = FeatureUnion([
                          #  ('dt_change', dt_tfr),
                           ('Ordinal',dest_tfr),
                           ('feat', cols_tfr)
                           ]).set_output(transform='pandas')


# the pipeline to complete above processing
pipe = Pipeline([('replace',mis_replace),
                 ('mfr_consolidate',mfr_consolidate),
                 ('feature_union', feat_union),
                 ('col_cleaner',column_cleaner)
                ])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


In [33]:
new_X = pipe.fit_transform(X)

Shape of df: (293218, 61)
Shape of tmp_air: (293217,)
Average of tmp_air: 192.2531879120242
Shape of tmp_air_d: (293109,)
Average of tmp_air_d: 161.6575471923414
Shape of tmp_dew: (293198,)
Average of tmp_dew: 113.17228971548236
Shape of tmp_dew_d: (293065,)
Average of tmp_dew_d: 81.1942606588982


In [38]:
new_X.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,293218.0,2.013852e+03,2.847053,2010.0,2011.0,2014.0,2016.0,2019.0
Quarter,293218.0,2.529350e+00,1.111700,1.0,2.0,3.0,4.0,4.0
Month,293218.0,6.636809e+00,3.373459,1.0,4.0,7.0,10.0,12.0
DayofMonth,293218.0,1.568119e+01,8.797893,1.0,8.0,16.0,23.0,31.0
DayOfWeek,293218.0,3.968835e+00,1.998879,1.0,2.0,4.0,6.0,7.0
OriginAirportSeqID,293218.0,1.129804e+06,1.334996,1129802.0,1129803.0,1129803.0,1129804.0,1129806.0
DestAirportSeqID,293218.0,1.295127e+06,159365.377716,1014001.0,1129202.0,1320402.0,1412202.0,1591904.0
Dest,293218.0,5.659051e+01,31.179556,0.0,28.0,61.0,81.0,105.0
DestStateFips,293218.0,2.761859e+01,16.598674,1.0,12.0,29.0,42.0,72.0
CRSElapsedTime,293218.0,1.447278e+02,62.662977,45.0,105.0,141.0,175.0,556.0


In [35]:
X_test = pipe.transform(test)

Shape of df: (56924, 61)
Shape of tmp_air: (56924,)
Average of tmp_air: 135.42303773452323
Shape of tmp_air_d: (56870,)
Average of tmp_air_d: 106.80465975030772
Shape of tmp_dew: (56924,)
Average of tmp_dew: 58.37746820321833
Shape of tmp_dew_d: (56862,)
Average of tmp_dew_d: 31.949210368963456


In [36]:
new_X.to_parquet(files['Prepro']['Train_Full_All'])

In [ ]:
# confirm file can be read in
df = pd.read_parquet(files['Final_Sets']['Train_All'])

In [37]:
# Save off test set
X_test.to_parquet(files['Final_Sets']['Test_set'])

(293218, 116)